In [42]:
# Holidays table for US + Canada

import pandas as pd
import holidays
from datetime import date

future_year = date.today().year + 10

# Parameters (edit as needed)
countries = ["US", "CA"]
years = list(range(2015, future_year))     # from 2015 -> current year + 10
observed = True                     # include “observed” shifted dates

In [43]:
state_provience_dict = {}

# The holidays package is organized so that every country has its own class inside the library.

for country in countries:
    # 1. Try to get the "class" from the holidays package
    #    Example: holidays.US or holidays.CA
    country_class = getattr(holidays, country, None)
    
    # 2. From that class, ask for the list of subdivisions (states/provinces)
    #    If no class is found, fall back to an empty list
    country_subdivisions = list(getattr(country_class, "subdivisions", []))
    
    # 3. Store the results in our dictionary.
    #    We always add [None] at the start to represent the national holidays.
    #    Then we add the sorted list of subdivisions.
    state_provience_dict[country] = [None] + sorted(country_subdivisions)

print(state_provience_dict.keys())
state_provience_dict

dict_keys(['US', 'CA'])


{'US': [None,
  'AK',
  'AL',
  'AR',
  'AS',
  'AZ',
  'CA',
  'CO',
  'CT',
  'DC',
  'DE',
  'FL',
  'GA',
  'GU',
  'HI',
  'IA',
  'ID',
  'IL',
  'IN',
  'KS',
  'KY',
  'LA',
  'MA',
  'MD',
  'ME',
  'MI',
  'MN',
  'MO',
  'MP',
  'MS',
  'MT',
  'NC',
  'ND',
  'NE',
  'NH',
  'NJ',
  'NM',
  'NV',
  'NY',
  'OH',
  'OK',
  'OR',
  'PA',
  'PR',
  'RI',
  'SC',
  'SD',
  'TN',
  'TX',
  'UM',
  'UT',
  'VA',
  'VI',
  'VT',
  'WA',
  'WI',
  'WV',
  'WY'],
 'CA': [None,
  'AB',
  'BC',
  'MB',
  'NB',
  'NL',
  'NS',
  'NT',
  'NU',
  'ON',
  'PE',
  'QC',
  'SK',
  'YT']}

In [44]:
rows = []

for country in countries:
    subdiv_list = state_provience_dict[country]  

    for subdiv in subdiv_list:
        stat_holiday_calendar = holidays.country_holidays(
            country, years=years, subdiv=subdiv, observed=observed
        )

        for date, name in stat_holiday_calendar.items():
            if subdiv is None:
                province_state = "n/a"
                provincial_state_holiday_flag = False
                national_holiday_flag = True
            else:
                province_state = subdiv
                provincial_state_holiday_flag = True
                national_holiday_flag = False

            rows.append({
                "date": pd.to_datetime(date),
                "country": country,
                "province/state": province_state,
                "provincial_state_holiday_flag": provincial_state_holiday_flag,
                "national_holiday_flag": national_holiday_flag,
                "holiday_description": name,
            })

# Create DataFrame
cols = [
    "date",
    "country",
    "province/state",
    "provincial_state_holiday_flag",
    "national_holiday_flag",
    "holiday_description",
]
stat_holiday_calendar = pd.DataFrame(rows, columns=cols)




stat_holiday_calendar.head(20)

,date,country,province/state,provincial_state_holiday_flag,national_holiday_flag,holiday_description
0,2015-01-01,US,n/a,False,True,New Year's Day
1,2015-05-25,US,n/a,False,True,Memorial Day
2,2015-07-04,US,n/a,False,True,Independence Day
3,2015-07-03,US,n/a,False,True,Independence Day (observed)
4,2015-09-07,US,n/a,False,True,Labor Day
5,2015-11-11,US,n/a,False,True,Veterans Day
6,2015-11-26,US,n/a,False,True,Thanksgiving Day
7,2015-12-25,US,n/a,False,True,Christmas Day
8,2015-01-19,US,n/a,False,True,Martin Luther King Jr. Day
9,2015-02-16,US,n/a,False,True,Washington's Birthday


In [45]:
print(stat_holiday_calendar['date'].max())
print(stat_holiday_calendar['date'].min())

2034-12-31 00:00:00
2015-01-01 00:00:00


In [46]:
stat_holiday_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18409 entries, 0 to 18408
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   date                           18409 non-null  datetime64[ns]
 1   country                        18409 non-null  object        
 2   province/state                 18409 non-null  object        
 3   provincial_state_holiday_flag  18409 non-null  bool          
 4   national_holiday_flag          18409 non-null  bool          
 5   holiday_description            18409 non-null  object        
dtypes: bool(2), datetime64[ns](1), object(3)
memory usage: 611.4+ KB


In [4]:
stat_holiday_calendar.to_csv(r"C:\Users\jverc\Downloads\stat_holiday_calendar.csv", index=False)